# Import packages

In [5]:
import pandas as pd
import numpy as np
from twitterscraper import query_tweets
import datetime as dt
#reading excel file and importing
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import TweetTokenizer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import pandas as pd
from matplotlib import pyplot as plt
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.dates as mdates
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import gensim
sns.set()
%matplotlib inline
from urllib.request import urlopen
import urllib.request
import builtins
import pickle
import urllib.request                   
from bs4 import BeautifulSoup           
from time import sleep
from random import uniform
from selenium import webdriver
import selenium
import time
from matplotlib.pyplot import subplots
from IPython.display import Image, display
tokenizer = TweetTokenizer()
stop_words = stopwords.words("english")
stop_words.extend(["&amp;", "&gt;", "&lt;"])
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Namee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
#reading pickeled news handles for fact based classification
#import pickle
with open("final_handle_list.pkl", "rb") as f:
    final_handle_list = pickle.load(f) 
#Reading pickeled word repositiories representing Databreach and Racial categories
with open("word_repositories.pkl", "rb") as f:
    word_repositories = pickle.load(f) 
#Reading pickled top hashtag list for each category
with open("classification_hashtags.pkl", "rb") as f:
    classification_hashtags = pickle.load(f) 

# Define Functions

#To fetch data from twitter using the handles and dates

In [15]:
#To fetch data from twitter using the handles and dates
def fetchData(issue,about,begin_date,end_date):
    issue=query_tweets(about,begindate=begin_date,enddate=end_date,limit=20000,lang='english')
    return issue

#Function to use when query is available directly

In [16]:
#function to use when query is available directly
def fetchQueryData(issue,query):
    issue=query_tweets(query)
    return issue

#To clean the data

In [ ]:
#To clean the data
def clean_tweets(tokenized_tweets):
    clean_text = []
    handles = []
    hashtags = []
    tweet_len = []
    for tweet in tokenized_tweets:
        thandles = []
        thashtags = []
        joined_words = " "
        words = [w.lower() for w in tweet if len(w)>2 and w not in stop_words]
        thandles = [w for w in words if re.search("^@\w+", w)]
        thashtags = [w for w in words if re.search("^#\w+", w)]
        words = [w for w in words if w not in thandles and w not in thashtags]
        words = [lemmatizer.lemmatize(w) for w in words]
        joined_words = joined_words.join(words)
        tlength = len(joined_words)
        handles.append(thandles)
        hashtags.append(thashtags)
        clean_text.append(joined_words)
        tweet_len.append(tlength)
    return clean_text,tweet_len,handles,hashtags

#Function to get the urls from string format of url

In [17]:
#function to get the urls from string format of url
def get_url(urllist):
    get_url=[]
    for url in urllist:
        try:
            res = urlopen(url)
            actual_url = res.geturl()
            get_url.append(actual_url)
        except:
            get_url.append(url)
    return get_url

#Function to get the url list to webscrape

In [25]:
#function to get the url list to webscrape     
def get_urllist(dataframe):
    valid_url_list=[]
    for tweet in tqdm(dataframe["text"]):
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet)
        for url in urls:
            for handle in final_handle_list:
                if(len(handle)!=0):
                    if handle.lower() in url.lower():
                        valid_url_list.append(url)
    return valid_url_list

#Function to get hashtag lists

In [19]:
#Function to get hashtag lists
def classify_hashtag(dataframe):
    klist=[]
    hashtag_list=[]
    list_of_hashtags=[]
    for i in dataframe['hashtags']:
            if len(i)>0:
                for element in i:
                    if element not in klist:
                        klist.append(element)
    hashtag_list.append(klist)       
    flat_list = [item for sublist in hashtag_list for item in sublist]
    dict = {} 
    for w in flat_list:
        dict[w] = dict.get(w, 0) + 1

    word_freq = []
    for key, value in dict.items():
        word_freq.append((value, key))
    word_freq.sort(reverse=True)
    values, labels = zip(*word_freq)
    list_of_hashtags=list(set(labels))
    return list_of_hashtags

    

#Function to check the category through webscraping

In [1]:
def webscrape_count(url_list):
    #Function to scrape th url
    Racial_count=0
    Databreachcount=0
    Other_count=0
    for url in url_list:
        driver = webdriver.Chrome("C://chromedriver_win32/chromedriver.exe")
        driver.implicitly_wait(25)
        try:     
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            text_string=" "
            for i in soup.find_all('p'):
                text_string=text_string+(i.text)
            for word in word_repositories["Databreach"]:
                if(word in text_string):
                    Databreachcount=Databreachcount+1
            for word in word_repositories["Racial"]:
                if(word in text_string):
                    Racial_count=Racial_count+1                   
        except:
            continue
        finally:
            driver.quit()
    print("Data Breach count:",Databreachcount,Racial_count)
    print("Racial count:",Racial_count)
    return Databreachcount,Racial_count